## 17단계: 메모리 관리와 순환 참조

> DeZero는 교육용이기 때문에 성능은 다소 희생하였습니다. \
하지만 처리 속도와 메모리 사용량에 전혀 신경 쓰지 않는 것도 교육적으로 좋지는 않을 것 같습니다. \
이번 단계에서는 메모리 관리를 개선해보겠습니다.

### 17.1 메모리 관리

파이썬은 필요 없어진 객체를 메모리에서 삭제해준다. \
파이썬(정확히는 CPython)의 메모리 관리 방식은 두 가지로 나뉘는데, 하나는 참조(reference) 수를 세는 방식이고, 다른 하나는 세대(generation)를 기준으로 쓸모 없어진 객체(garbage)를 회수(collection)하는 방식이다. \
이 책에서는 전자를 '참조 카운트'로, 후자를 'GC(Garbage Collection)'이라 부를 것이다.

> 문헌에 따라 참조 카운트 방식의 메모리 관리도 GC의 일부로 보기도 한다.


### 17.2 참조 카운트 방식의 메모리 관리

참조 카운트는 구조가 간단하고 속도가 빠르다. \
모든 객체는 생성될 때 참조 카운트가 0으로 초기화되고, 다른 객체가 참조할 때마다 1씩 증가하고, 참조가 끊길 때마다 1씩 감소하여 다시 0이 되면 회수된다. \
참조 카운트가 증가하는 케이스의 예시는 다음과 같다.
- 대입 연산자를 사용할 때
- 함수에 인수로 전달할 때
- 컨테이너 타입 객체(리스트 튜플, 클래스 등)에 추가할 때

In [2]:
# 참조 카운트 의사코드 1

class obj:
    pass

def f(x):
    print(x)

a = obj()  # 변수에 대입: 참조 카운트 1
f(a)  # 함수에 전달: 함수 내에서는 참조 카운트 2
# 함수 완료: 참고 카운트 1
a = None  # 변수 대입 해제: 참조 카운트 0 -> 회수(collect)

In [ ]:
# 참조 카운트 의사코드 2

a = obj()  # 변수 대입: a.참조 1
b = obj()  # 변수 대입: b.참조 1
c = obj()  # 변수 대입: c.참조 1

a.b = b  # a의 속성에 대입: b.참조 2
b.c = c  # b의 속성에 대입: c.참조 2
# --- [그림 17-1]의 첫번째 그림

a = b = c = None  # 변수 할당 해제: a.참조 0, b.참조 1, c.참조 1
# --- [그림 17-1]의 두번째 그림

# a.참조 0 -> a 회수(collect) -> b.참조 0, c.참조 1
# b.참조 0 -> b 회수(collect) -> c.참조 0
# c.참조 0 -> c 회수(collect)

<img src="images/그림 17-1.png" width=550/>

### 17.3 순환 참조

먼저 의사코드를 통해 상황을 살펴보자.

In [5]:
# 참조 카운트 의사코드 3

a = obj()  # 변수 대입: a.참조 1
b = obj()  # 변수 대입: b.참조 1
c = obj()  # 변수 대입: c.참조 1

a.b = b  # a의 속성에 대입: b.참조 2
b.c = c  # b의 속성에 대입: c.참조 2
c.a = a  # c의 속성에 대입: a.참조 2
# --- [그림 17-2]의 첫번째 그림

a = b = c = None  # 변수 할당 해제: a.참조 1, b.참조 1, c.참조 1
# --- [그림 17-2]의 두번째 그림

<img src="images/그림 17-2.png" width=550/>

객체 a, b, c는 서로를 참조하기 때문에 자동으로 메모리에서 회수되지 않는다. 이를 해결하기 위해 GC(정확히는 '세대별 가비지 컬렉션(generational garbage collection)')가 등장한다.

GC는 더욱 영리한 방식으로 불필요한 객체를 찾아낸다. (구조 설명은 생략) GC는 참조 카운트와 달리 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출된다. 물론 `gc`모듈을 임포트하여 `gc.collect()`를 실행하여 명시적으로 호출할 수도 있다.

하지만 GC에 메모리 관리를 일임하면 순환 참조로 인해 메모리 사용량이 커질 수 있는데, 머신러닝이나 딥러닝에서 메모리는 중요한 자원이므로 순환 참조를 만들지 않는 것이 좋다.

다시 DeZero로 돌아와서, 현재 DeZero에는 사실 '변수'와 '함수'를 연결하는 지점에 순환 참조가 숨어 있다. 이를 파이썬의 약한 참조(weak reference) 모듈인 `weakref`로 해결해보자.

<img src="images/그림 17-3.png" width=350/>

### 17.4 weakref 모듈

`weakref` 모듈은 다른 객체를 참조하되 참조 카운트를 증가시키지 않는 여러 기능들을 지원한다.

In [ ]:
import weakref
import numpy as np

a = np.array([1, 2, 3])  # a 변수에 할당: ndarray 객체 참조 카운트 1
b = weakref.ref(a)  # weakref로 b에 할당: ndarray 객체 참조 카운트 1 (변화 없음)

print(repr(b))

print(b())

<weakref at 0x7f9fe2e48ae0; to 'numpy.ndarray' at 0x7f9fe2eb17d0>
[1 2 3]


`weakref.ref` 함수를 사용하면 원래 변수를 가리키는 약한 참조를 만들 수 있다. \
이 때 참조되는 객체에 접근하려면 호출('call')하여 불러올 수 있다.

In [ ]:
a = None  # 변수 할당 해제: ndarray 객체 참조 카운트 0
# ndarray 객체 삭제
b

<weakref at 0x7f9fe2e48ae0; dead>

ndarray 객체가 삭제되면 변수 b에서도 객체를 참조할 수 없게 된다.

만약 `weakref.ref`의 함수 호출 방식이 불편하다면 `weakref.proxy`를 사용할 수도 있다.

In [ ]:
a = np.array([1, 2, 3])
b = weakref.proxy(a)

b.shape  # ref와 다르게 proxy는 바로 원본 객체의 속성과 메서드에 접근할 수 있다.

(3,)

책과 다르게 DeZero에서는 `weakref.proxy`를 도입해보자.

In [16]:
import weakref
from heapq import heappush, heappop


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        
        # self.outputs에 약한 참조 적용
        # self.outputs = outputs  # 기존 코드
        # self.outputs = [weakref.ref(output) for output in outputs]  # 책 버전 수정 코드
        self.outputs = [weakref.proxy(output) for output in outputs]  # weakref.proxy 적용 코드
        
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()



class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs_heap = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                heappush(funcs_heap, f)
                seen_set.add(f)
        
        add_func(self.creator)
        
        while funcs_heap:
            f = heappop(funcs_heap)
            # output 참조 시 약한 참조 적용
            # gys = [output.grad for output in f.outputs]  # 기존 코드
            # gys = [output().grad for output in f.outputs]  # 책 버전 수정 코드
            gys = [output.grad for output in f.outputs]  # weakref.proxy 적용 코드
            
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
    
    def cleargrad(self):
        self.grad = None

### 17.5 동작 확인

In [17]:
# 필요 모듈 정의

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

In [18]:
for i in range(10):
    x = Variable(np.random.randn(10000))  # 거대한 데이터
    y = square(square(square(x)))  # 복잡한 계산

위 반복문은 다음과 같이 복잡한 참조 구조를 반복하여 생성한다.

<img src="images/그림 17-4.png" width=550/>

예전 코드라면 순환참조가 발생하여 모든 객체가 메모리에 남아있었겠지만, 현재는 순환참조가 없어졌기 때문에 객체가 메모리에서 잘 회수된다. 이는 외부 라이브러리인 'memory profiler' 등을 사용하면 측정할 수 있다.